# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,boguchany,58.3814,97.4531,-15.08,98,40,0.89,RU,1699496265
1,1,degerfors,59.2380,14.4308,3.71,95,100,2.32,SE,1699496265
2,2,yellowknife,62.4560,-114.3525,-7.71,93,75,1.54,CA,1699496267
3,3,rio negrinho,-26.2544,-49.5183,21.08,85,100,0.32,BR,1699496267
4,4,olonkinbyen,70.9221,-8.7187,-0.48,74,98,4.25,SJ,1699496268


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_ideal = city_data_df.loc[(city_data_df['Max Temp'] <= 26) & (city_data_df['Max Temp'] > -10) & (city_data_df['Humidity'] <70)]

# Drop any rows with null values
city_data_df_ideal_dropna = city_data_df_ideal.dropna()

# Display sample data
city_data_df_ideal_dropna.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,kodinar,20.7903,70.7031,25.87,66,0,2.03,IN,1699496269
7,7,nova sintra,14.8667,-24.7167,22.92,67,92,7.43,CV,1699496269
10,10,papatowai,-46.5619,169.4708,11.57,54,39,5.05,NZ,1699496271
11,11,invercargill,-46.4000,168.3500,12.23,64,35,4.02,NZ,1699496272
18,18,albany,42.6001,-73.9662,2.52,65,100,0.71,US,1699496018


### Step 3: Create a new DataFrame called `hotel_df`.

In [3]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ' '

# Display sample data
hotel_df.head()

C:\Users\natha\AppData\Local\Temp\ipykernel_11712\229348391.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ' '


,City,Country,Lat,Lng,Humidity,Hotel Name
0,boguchany,RU,58.3814,97.4531,98,
1,degerfors,SE,59.2380,14.4308,95,
2,yellowknife,CA,62.4560,-114.3525,93,
3,rio negrinho,BR,-26.2544,-49.5183,85,
4,olonkinbyen,SJ,70.9221,-8.7187,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [5]:
# Set parameters to search for a hotel

categories = "accommodation.hotel"
radius = 10000

params = {
    "categories":categories,
    "apiKey":geoapify_key    
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df['Lat'].iloc[index]
    longitude = hotel_df['Lng'].iloc[index]
       
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
boguchany - nearest hotel: Полет
degerfors - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
rio negrinho - nearest hotel: Hotel João de Barro
olonkinbyen - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
kodinar - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
khandyga - nearest hotel: Апельсин
edinburgh of the seven seas - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
shantobe - nearest hotel: No hotel found
parral - nearest hotel: Hotel Adriana
pacific grove - nearest hotel: Pacific Grove Inn
mejit - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
hermanus - nearest hotel: Aloe guest house
albany - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
chamical - nearest hotel: No hotel found
port-gentil - nearest hotel: Tara-Mé
kodiak - nearest hotel: Shelikof Lod

,City,Country,Lat,Lng,Humidity,Hotel Name
0,boguchany,RU,58.3814,97.4531,98,Полет
1,degerfors,SE,59.2380,14.4308,95,No hotel found
2,yellowknife,CA,62.4560,-114.3525,93,No hotel found
3,rio negrinho,BR,-26.2544,-49.5183,85,Hotel João de Barro
4,olonkinbyen,SJ,70.9221,-8.7187,74,No hotel found
...,...,...,...,...,...,...
547,saudarkrokur,IS,65.7461,-19.6394,88,Mikligarður
548,springs,ZA,-26.2500,28.4000,93,Manhattan Hotel & Action Bar
549,mirny,RU,62.5353,113.9611,77,Зарница
550,ulaangom,MN,49.9811,92.0667,93,АНДУУД


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.01,
    color = "City",
    hover_cols = ['Hotel Name','Country']
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)